you can dynamically share data between these two routes, but Flask does not automatically persist data between different requests. You have a few options to pass the stock_videos and stock_audio data from /prompt_tab to /regenerate_storyboard_tab:

sample code

Yes, we can dynamically share data between these two routes, but Flask does not automatically persist data between different requests. You have a few options to pass the `stock_videos` and `stock_audio` data from `/prompt_tab` to `/regenerate_storyboard_tab`:  

---

### **1️⃣ Store Data in a Global Cache (Flask `g` or `session`)**  
You can store the data in Flask's `g` (global object) or `session` if it’s a per-user session.  

#### **Example Using Flask `g` (Only Works Within the Same Request)**
```python
from flask import g

@app.route("/prompt_tab", methods=["POST"])
@jwt_required()
def prompt_tab_starts():
    try:
        user_id = get_jwt_identity()
        stock_media_str = request.form.get("stock_media", "[]")  
        stock_media = json.loads(stock_media_str) if stock_media_str else []

        stock_videos = []
        stock_audio = []

        for media in stock_media:
            file_type = media.get("file_type")
            file_s3_path = media.get("file_s3_path")
            if file_type in ["AUDIO_VIDEO", "VIDEO"]:
                stock_videos.append({"path": file_s3_path})
            elif file_type == "AUDIO":
                stock_audio.append({"path": file_s3_path})

        g.stock_videos = stock_videos
        g.stock_audio = stock_audio
        
        return jsonify({"message": "Data stored in g"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/regenerate_storyboard_tab", methods=["POST"])
@jwt_required()
def regenerate_button_storyboard():
    try:
        stock_videos = getattr(g, "stock_videos", [])
        stock_audio = getattr(g, "stock_audio", [])

        return jsonify({"stock_videos": stock_videos, "stock_audio": stock_audio})
    except Exception as e:
        return jsonify({"error": str(e)}), 500
```
⚠ **Limitation**: `g` only works for a single request lifecycle. It won’t persist across different API calls.

---

### **2️⃣ Store Data in Redis or Database**
Since the data needs to be accessed in another API request, **storing it in Redis (preferred) or MongoDB** is a good approach.

#### **Example Using Redis**
```python
import json
import redis

redis_client = redis.StrictRedis(host='localhost', port=6379, db=0, decode_responses=True)

@app.route("/prompt_tab", methods=["POST"])
@jwt_required()
def prompt_tab_starts():
    try:
        user_id = get_jwt_identity()
        stock_media_str = request.form.get("stock_media", "[]")  
        stock_media = json.loads(stock_media_str) if stock_media_str else []

        stock_videos = []
        stock_audio = []

        for media in stock_media:
            file_type = media.get("file_type")
            file_s3_path = media.get("file_s3_path")
            if file_type in ["AUDIO_VIDEO", "VIDEO"]:
                stock_videos.append({"path": file_s3_path})
            elif file_type == "AUDIO":
                stock_audio.append({"path": file_s3_path})

        redis_client.set(f"stock_videos:{user_id}", json.dumps(stock_videos))
        redis_client.set(f"stock_audio:{user_id}", json.dumps(stock_audio))

        return jsonify({"message": "Data stored in Redis"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/regenerate_storyboard_tab", methods=["POST"])
@jwt_required()
def regenerate_button_storyboard():
    try:
        user_id = get_jwt_identity()
        stock_videos = json.loads(redis_client.get(f"stock_videos:{user_id}") or "[]")
        stock_audio = json.loads(redis_client.get(f"stock_audio:{user_id}") or "[]")

        return jsonify({"stock_videos": stock_videos, "stock_audio": stock_audio})
    except Exception as e:
        return jsonify({"error": str(e)}), 500
```
✅ **Pros**:
- Works across different API calls
- Fast retrieval
- Data is per-user (each user has their own cache)

---

### **3️⃣ Store in MongoDB**
If you prefer database storage, you can store the `stock_videos` and `stock_audio` inside MongoDB.

#### **Example Using MongoDB**
```python
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["media_cache"]

@app.route("/prompt_tab", methods=["POST"])
@jwt_required()
def prompt_tab_starts():
    try:
        user_id = get_jwt_identity()
        stock_media_str = request.form.get("stock_media", "[]")  
        stock_media = json.loads(stock_media_str) if stock_media_str else []

        stock_videos = []
        stock_audio = []

        for media in stock_media:
            file_type = media.get("file_type")
            file_s3_path = media.get("file_s3_path")
            if file_type in ["AUDIO_VIDEO", "VIDEO"]:
                stock_videos.append({"path": file_s3_path})
            elif file_type == "AUDIO":
                stock_audio.append({"path": file_s3_path})

        db.stock_data.update_one({"user_id": user_id}, 
                                 {"$set": {"stock_videos": stock_videos, "stock_audio": stock_audio}}, 
                                 upsert=True)

        return jsonify({"message": "Data stored in MongoDB"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/regenerate_storyboard_tab", methods=["POST"])
@jwt_required()
def regenerate_button_storyboard():
    try:
        user_id = get_jwt_identity()
        user_data = db.stock_data.find_one({"user_id": user_id}) or {}

        stock_videos = user_data.get("stock_videos", [])
        stock_audio = user_data.get("stock_audio", [])

        return jsonify({"stock_videos": stock_videos, "stock_audio": stock_audio})
    except Exception as e:
        return jsonify({"error": str(e)}), 500
```
✅ **Pros**:
- Persistent storage
- Good for logging and analytics
- No risk of data loss after a server restart

---

### **Which Approach is Best?**
| **Method**          | **Pros**                                      | **Cons**                                 |
|---------------------|---------------------------------------------|-----------------------------------------|
| `g` (Flask Global) | Simple, no extra dependencies               | Data is lost after request ends        |
| **Redis** (Recommended) | Fast, efficient, survives app restarts    | Requires Redis setup                   |
| **MongoDB**         | Permanent storage, scalable                 | Slightly slower than Redis             |

🚀 **Use Redis if you need fast temporary storage and MongoDB if you need long-term persistence.**